In [ ]:
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import requests
import json
import pprint
#key file from service instance is necessary to login
with open("ai-core-key.txt") as file:
    aicorekey = json.load(file)

In [ ]:
AUTH_URL = aicorekey["url"]
AI_API = aicorekey["serviceurls"]["AI_API_URL"] + "/v2"
CLIENT_ID = aicorekey["clientid"]
CLIENT_SECRET = aicorekey["clientsecret"]
RESOURCE_GROUP = "llm"
SCENARIO_ID = "transformers"
SERVING_EXECUTABLE = "transformers"

In [ ]:
ai_api_v2_client = AIAPIV2Client(
    base_url=AI_API + "/lm", 
    auth_url=AUTH_URL + "/oauth/token", 
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET, 
    resource_group=RESOURCE_GROUP
)

Evaluate Tasks

In [ ]:
def get_response(full_prompt, args={}):
    res = requests.post(
        f"https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd57ca34bd22438c/v2/predict",
        json={"prompt": full_prompt, "args": args},
        headers={
            "Authorization": ai_api_v2_client.rest_client.get_token(),
            "ai-resource-group": RESOURCE_GROUP,
            "Content-Type": "application/json"
        })
    if res.status_code != 200:
        raise Exception("ERROR WITH DEPLOYMENT " + str(res.status_code) + " " + str(res.content))
    return res.json()[0]["generated_text"][len(full_prompt):]

In [ ]:
def build_llama2_prompt(role_prompt, task_prompt):
    B_S, E_S = "<s>", " </s>"
    B_INST, E_INST = "[INST]", " [/INST]\n"
    B_SYS, E_SYS = " <<SYS>>\n", "\n<</SYS>>\n\n"
    SYSTEM_PROMPT = B_SYS + role_prompt + E_SYS
    return B_S + B_INST + SYSTEM_PROMPT + task_prompt + E_INST

In [ ]:
print(get_response(build_llama2_prompt(role_prompt="""This app classifies text messages as either "Positive", "Neutral" or "Negative". You recieve 3 examples of the format and classify the fourth. Only respons with one word.""", task_prompt="""
###
Text: I really like your shoes. Are they new?
Sentiment: Positive
###
Text: Peter is very stupid.
Sentiment: Negative
###
Text: I first met her last week. She is super nice.
Sentiment: Positive
###
Text: The implementation of SAP's new solution significantly improved our business processes and efficiency.

In [ ]:
# generated reviews for test purposes

positive_reviews = [
    "Impressed with this mobile data plan! The data speed is great, and I can seamlessly stream my favorite shows and movies without buffering. Plus, the pricing is reasonable, making it a good value for money. Recommended!",
    "Switched to this data plan, coverage improved a lot. Works even in remote areas, which is essential for my job. The signal strength is consistent. Thumbs up!",
    "Unlimited data is a lifesaver! No more worries about running out of data or facing overage charges. It's a huge relief for my busy lifestyle.",
    "This data plan offers fantastic flexibility! Customizing my data package based on my needs each month is easy and convenient. Love having control over my plan.",
    "The data rollover feature is a game-changer! No more wasted data. I can roll over unused data to the next month. It's brilliant!",
    "Reliable mobile data plan. Consistently get a strong signal no matter the time or location. No interruptions during browsing or streaming.",
    "The mobile app is a lifesaver. It's user-friendly and allows me to monitor my data usage effectively. Real-time alerts are helpful.",
    "Loyalty rewards program is a fantastic perk. Earning points for every payment and redeeming them for discounts or free data is a nice touch.",
    "Works smoothly abroad! No need for expensive international roaming packages. Great convenience for travelers."
]

negative_reviews = [
    "Signal drops occasionally, and it's annoying. It can happen in unexpected areas, which is disappointing.",
    "Overpriced for the service provided. While the speed is okay, it doesn't justify the cost compared to other options.",
    "Customer service needs improvement. Had issues with billing and it was frustrating to resolve.",
    "Data caps are limiting. It's inconvenient to run out of data before the month ends.",
    "Coverage in rural areas is lacking. Struggled to get a reliable signal during travels.",
    "Billing issues were confusing. Unexpected charges needed clarification.",
    "Mobile app has glitches. Inaccurate data usage display and notifications are a hassle.",
    "Limited international options. Had difficulties using the data plan abroad.",
    "Data speed slows during peak hours. Inconsistent performance can be frustrating."
]

In [ ]:
[print(get_response(build_llama2_prompt(role_prompt="""This app classifies text reviews about a mobile carrier as either "Positive" or "Negative". You recieve 3 examples of the format and classify the fourth. Only respons with one word.""", task_prompt=f"""
###
Text: Customer service is outstanding! Had a billing issue, and the representatives were helpful and patient. I appreciate the support received.
Sentiment: Positive
###
Text: Mixed feelings about this data plan. The speed fluctuates at times.
Sentiment: Negative
###
Text: The callcenter responded quickly and solved my issue.
Sentiment: Positive
###
Text: {review}
Sentiment: """))) for review in positive_reviews];

In [ ]:
[print(get_response(build_llama2_prompt(role_prompt="""This app classifies text reviews about a mobile carrier as either "Positive" or "Negative". You recieve 3 examples of the format and classify the fourth. Only respons with one word.""", task_prompt=f"""
###
Text: Customer service is outstanding! Had a billing issue, and the representatives were helpful and patient. I appreciate the support received.
Sentiment: Positive
###
Text: Mixed feelings about this data plan. The speed fluctuates at times.
Sentiment: Negative
###
Text: The callcenter responded quickly and solved my issue.
Sentiment: Positive
###
Text: {review}
Sentiment: """))) for review in negative_reviews];

In [ ]:
[print(get_response(build_llama2_prompt(role_prompt="""This app extracts information from text reviews about a mobile carrier. Please extract positive keywords from the review. You recieve 3 examples of the format and fill the fourth. Only respons with one line.""", task_prompt=f"""
###
Text: Customer service is outstanding! Had a billing issue, and the representatives were helpful and patient. I appreciate the support received.
Positive Keywords: customer service, billing, issue
###
Text: Mixed feelings about this data plan. The speed fluctuates at times.
Positive Keywords: 
###
Text: The callcenter responded quickly and solved my issue.
Positive Keywords: callcenter, quick
###
Text: {review}
Positive Keywords: """))) for review in negative_reviews];

In [ ]:
[print(f"""This app extracts insights from customer reviews of a mobile carrier. 
       Please extract rate the review in the categories Internet Speed, Customer Service, Price, Sales Team as either "Positive", "Neutral" or "Negative". 
       You recieve 3 examples of the format and fill the fourth.
       Most of the time the categories are neutral, only put a judgement if it is clearly referenced in the text.
###
Review: Customer service is outstanding! Had a billing issue, and the representatives were helpful and patient. I appreciate the support received.
Internet Speed: Neutral
Customer Service: Positive
Price: Neutral
Sales Team: Neutral
###
Review: Mixed feelings about this data plan. The speed fluctuates at times.
Internet Speed: Negative
Customer Service: Neutral
Price: Neutral
Sales Team: Neutral
###
Review: The callcenter responded quickly and solved my issue.
Internet Speed: Neutral
Customer Service: Positive
Price: Neutral
Sales Team: Neutral
###
Review: {review}
""") for review in negative_reviews]